In [ ]:
from ragas.testset import TestsetGenerator
from langchain_openai import OpenAIEmbeddings,ChatOpenAI 
from ragas.embeddings import HuggingFaceEmbeddings
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers import SingleHopSpecificQuerySynthesizer, MultiHopSpecificQuerySynthesizer
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
os.environ['OPENAI_API_KEY'] = "sk-7695ab4ed4f249a4aa5a3f63dd8b1d1a"
os.environ['OPENAI_API_BASE'] = "https://api.deepseek.com"

from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings as BaseEmbeddings
from typing import List

class SentenceTransformerEmbeddings(BaseEmbeddings):
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, convert_to_tensor=False).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode([text], convert_to_tensor=False).tolist()[0]

# 使用示例
texts = ["This is a test sentence.", "This is another test sentence."]
emb = HuggingFaceEmbeddings(model="BAAI/bge-m3")
generator_llm = ChatOpenAI(model = 'deepseek-chat')
critic_llm = ChatOpenAI(model = 'deepseek-chat')
# 修改生成器创建部分
generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=emb
)
from langchain.document_loaders import TextLoader
from langchain.schema import Document
from pathlib import Path
from langchain.schema import Document



# 指定txt文件的路径
file_path = 'data/HSR-JR615-2000_En.txt'

# 使用TextLoader加载txt文件
loader = TextLoader(file_path,encoding='utf-8')

# 加载文档
documents = loader.load()
# 尝试不同的参数名称
testset = generator.generate(
    testset_size=10,  # 而非 test_size
    num_personas=3,  # 可选，默认就是 3
    batch_size=None,  # 可选，根据需要设定
    callbacks=None,
    token_usage_parser=None,
    with_debugging_logs=False,
    raise_exceptions=True,
    return_executor=False,
)


In [ ]:

input_file = 'input.csv'
output_file = 'output.csv'


In [1]:
import pandas as pd
import json
import re
from pathlib import Path
from typing import Any, List, Dict
from tqdm import tqdm

# 输入与输出
chunks_csv = "D:/MAKG_demo-master/MAKG_demo-master/RAG/data/semantic_chunks.filtered.csv"
kg_csv = "D:/MAKG_demo-master/MAKG_demo-master/RAG/data/knowledge_graph.csv"
output_jsonl = "D:/MAKG_demo-master/MAKG_demo-master/RAG/data/qa_pairs.jsonl"

# 读取分块与KG
chunks_df = pd.read_csv(chunks_csv)
kg_df = pd.read_csv(kg_csv)

# 构建 chunk_id -> triples 列表映射
chunk_to_triples: Dict[int, List[Dict[str, str]]] = {}
for _, row in kg_df.iterrows():
    cid = row.get("chunk_id", None)
    if pd.isna(cid):
        continue
    try:
        cid_int = int(cid)
    except Exception:
        continue
    triple = {
        "head": str(row.get("node_1", "")).strip(),
        "relation": str(row.get("edge", "")).strip(),
        "tail": str(row.get("node_2", "")).strip(),
    }
    if not triple["head"] or not triple["relation"] or not triple["tail"]:
        continue
    chunk_to_triples.setdefault(cid_int, []).append(triple)

# 取出文本与来源
chunks = chunks_df["text"].tolist()
sources = chunks_df["source"].tolist() if "source" in chunks_df.columns else [""] * len(chunks)

# 生成目标数量配置
total_target = 3  # 目标总问答对数，可按需调整
per_chunk = max(1, total_target // max(1, len(chunks)))
remainder = total_target % max(1, len(chunks))

# 简单选择顺序索引（可替换为抽样）
indices = list(range(len(chunks)))


# 一个最小的聊天调用封装，返回字符串
from typing import Optional
import os
from litellm import completion as llm_completion

DEFAULT_MODEL = "deepseek-chat"
BASE_URL = "https://api.deepseek.com/v1"
API_KEY = "sk-88cea9654d1f43328b373fea4774a0e8"

def llm_chat(messages: List[Dict[str, str]], model: Optional[str] = None, temperature: float = 0.0, **kwargs) -> str:
    m = model or DEFAULT_MODEL
    try:
        resp = llm_completion(model=m, messages=messages, api_base=BASE_URL, api_key=API_KEY, temperature=temperature, **kwargs)
        # 解析 OpenAI 风格返回
        content = None
        if isinstance(resp, dict):
            content = resp.get("choices", [{}])[0].get("message", {}).get("content", "")
        else:
            choices = getattr(resp, "choices", [])
            if choices:
                msg = getattr(choices[0], "message", None)
                if msg:
                    try:
                        content = msg.get("content")
                    except Exception:
                        content = getattr(msg, "content", "")
        return (content or "").strip()
    except Exception:
        return ""

# Prompt 生成

def _build_prompt(content: str, k: int) -> str:
    return (
        "You are an industrial document annotation assistant. Please generate high-quality English Q&A pairs based on the provided content.\n"
        "Strict Requirements:\n"
        "1. All questions and answers must be strictly based on the provided content, without introducing external knowledge;\n"
        "2. Each data item should be output as an object with fields: user_input (concise and natural English user question), reference_contexts (key sentences selected from the content), reference (detailed and content-rich English answer based on these contexts);\n"
        "3. The reference must be detailed, content-rich, and directly supported by reference_contexts;\n"
        "4. If the content is too brief, fragmented, or insufficient to form complete Q&A pairs (such as simple list items or phrases), do not generate any Q&A pairs;\n"
        "5. Avoid generating Q&A pairs from overly brief content (like simple entries such as '(5) Maintenance operations must be performed with power on...)';\n"
        "6. Output strictly as a JSON array, without any extra text or code blocks.\n\n"
        "Quality Requirements:\n"
        "- user_input should be concise and natural, like questions real users would ask\n"
        "- reference should be content-rich and detailed, providing complete explanations\n"
        "- Only generate high-quality, substantive Q&A pairs\n\n"
        f"Please generate 1 to {k} high-quality data items. If the content does not meet requirements, output an empty array [].\n\n"
        f"Content:\n{content}\n\n"
        "Output only JSON array:"
    )

# 主循环：根据分块内容生成问答对
results: List[Dict[str, Any]] = []
remaining = total_target
triples_per_doc = 3

for idx, i in enumerate(tqdm(indices, desc="Q&A pairs Generating")):
    if remaining <= 0:
        break

    txt = (chunks[i] or "").strip()
    src = sources[i] if i < len(sources) else ""

    # 拼接少量三元组，帮助关注关键信息
    triples_txt = ""
    tri_list = chunk_to_triples.get(i, [])
    if tri_list:
        lines: List[str] = []
        for t in tri_list[: max(1, triples_per_doc)]:
        # for t in tri_list:
            h = str(t.get("head", "")).strip()
            r = str(t.get("relation", "")).strip()
            ta = str(t.get("tail", "")).strip()
            if h and r and ta:
                lines.append(f"- ({h}) --{r}--> ({ta})")
        if lines:
            triples_txt = "\n\nKey Triples:\n" + "\n".join(lines)

    # content = txt + triples_txt
    content = txt

    # 分配数量
    k_this = per_chunk + (1 if idx < remainder else 0)
    k_this = min(k_this, remaining)
    if k_this <= 0:
        continue

    prompt = _build_prompt(content, k_this)
    resp = llm_chat(
        messages=[
            {"role": "system", "content": "Output only a JSON array, with each element containing user_input, reference_contexts, and reference."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )

    # 解析
    arr: List[Any] = []
    try:
        m = re.search(r"\[.*\]", resp, flags=re.S)
        arr = json.loads(m.group(0) if m else resp)
    except Exception:
        arr = []

    # 清洗并收集
    for obj in arr:
        if remaining <= 0:
            break
        if not isinstance(obj, dict):
            continue
        q = str(obj.get("user_input", "")).strip()
        rc = obj.get("reference_contexts", [])
        if isinstance(rc, str):
            rc = [rc]
        if not isinstance(rc, list):
            rc = []
        rc_clean = []
        for x in rc:
            sx = str(x).strip()
            if sx:
                rc_clean.append(sx)
        rc_clean = rc_clean[:3]
        ans = str(obj.get("reference", "")).strip()
        if not q or not ans or not rc_clean:
            continue
        results.append({
            "user_input": q,
            "reference_contexts": rc_clean,
            "reference": ans,
            "source": src,
            "chunk_id": i,
        })
        remaining -= 1

# 保存到 JSONL
outp = Path(output_jsonl)
outp.parent.mkdir(parents=True, exist_ok=True)
with outp.open("w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print({"ok": True, "count": len(results), "out_path": str(outp)})


Q&A pairs Generating:   0%|          | 0/554 [00:33<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# 覆盖/强化版 llm_chat：修复 litellm 需要明确 model 的问题
from typing import Optional, List, Dict
import os
from litellm import completion as llm_completion

# 健壮的默认值（环境变量若为空字符串则回退）
DEFAULT_MODEL = (os.environ.get("LITELLM_MODEL") or os.environ.get("OPENAI_MODEL") or "deepseek-chat")
BASE_URL = (os.environ.get("OPENAI_API_BASE") or "https://api.deepseek.com")
API_KEY = (os.environ.get("OPENAI_API_KEY") or "")


def llm_chat(messages: List[Dict[str, str]], model: Optional[str] = None, temperature: float = 0.0, **kwargs) -> str:
    """使用 litellm 调用，messages 采用 OpenAI 风格字典列表。
    可通过传入 model 覆盖，或设置环境变量 LITELLM_MODEL/OPENAI_MODEL。
    """
    m = (model or DEFAULT_MODEL)
    if not m:
        raise ValueError("litellm: 缺少模型名，请设置环境变量 LITELLM_MODEL 或传入 model 参数")
    try:
        resp = llm_completion(
            model=m,
            messages=messages,
            api_base=BASE_URL,
            api_key=API_KEY,
            temperature=temperature,
            **kwargs,
        )
        # 解析 OpenAI 风格返回
        content = None
        if isinstance(resp, dict):
            content = resp.get("choices", [{}])[0].get("message", {}).get("content", "")
        else:
            choices = getattr(resp, "choices", [])
            if choices:
                msg = getattr(choices[0], "message", None)
                if msg:
                    try:
                        content = msg.get("content")
                    except Exception:
                        content = getattr(msg, "content", "")
        return (content or "").strip()
    except Exception as e:
        # 返回空字符串避免中断主流程，也可改为 raise e 直接抛出
        return ""
